Some of the imports are not useful yet

In [1]:
from __future__ import print_function
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle 
from yaafelib import *

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Download GTZAN dataset once

In [3]:
def downloadGTZAN():
    filename = 'genres.tar.gz'
    if not os.path.exists(filename):
        filename, _ = urlretrieve('http://opihi.cs.uvic.ca/sound/genres.tar.gz', filename)
    else:
        print('File ' + filename + ' exists')
    
    return filename
filename = downloadGTZAN()

Extract files from .tar.gz

In [4]:
num_classes = 10

def extract(filename):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root):
        print('File %s already extracted' % (root))
    else:
        print('Extracting %s ' % filename)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))
                    if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
        raise Exception('Expected %d folders not found.' % (num_classes))
    print(data_folders)
    return data_folders

folders = extract(filename)

Extracting genres.tar.gz 
['genres/blues', 'genres/classical', 'genres/country', 'genres/disco', 'genres/hiphop', 'genres/jazz', 'genres/metal', 'genres/pop', 'genres/reggae', 'genres/rock']


In [ ]:
def get_features(audio_file):
    print('Getting features from ' + audio_file)
    fp = FeaturePlan(sample_rate=22050, normalize=1)  # sample rate of the .wav files TODO:check if only this
    # Features that seems to be most often used, so they are good to start with.
    fp.addFeature('mfcc: MFCC')
    fp.addFeature('zcr: ZCR')
    fp.addFeature('spectral_shape: SpectralShapeStatistics')
    fp.addFeature('lsf: LSF')
    fp.addFeature('magnitudeSpectrum: MagnitudeSpectrum')
    df = fp.getDataFlow()
    engine = Engine()
    engine.load(df)
    afp = AudioFileProcessor()
    afp.setOutputFormat('csv', 'output', {'Precision': '8', 'Metadata': 'False'})
    afp.processFile(engine, audio_file)
    engine.flush()


# get_features('/data/team-project/genres/rock/rock.00050.wav')

dataset = '/data/team-project/genres/'
for genre in os.listdir(dataset):
    for sample in os.listdir(dataset + genre):
        if sample.endswith('.wav'):
            absPath = dataset + genre + '/' + sample
            get_features(absPath)
